- This is done for practice purpose

In [2]:
import pandas as pd
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import StandardScaler, LabelEncoder, OneHotEncoder
from sklearn.pipeline import Pipeline
from scikeras.wrappers import KerasClassifier
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.callbacks import EarlyStopping
import pickle

In [3]:
data=pd.read_csv('Churn_Modelling.csv')
data = data.drop(['RowNumber', 'CustomerId', 'Surname'], axis=1)

label_encoder_gender = LabelEncoder()
data['Gender'] = label_encoder_gender.fit_transform(data['Gender'])

one_hot_encoder_geo = OneHotEncoder(handle_unknown='ignore')
geo_encoded = one_hot_encoder_geo.fit_transform(data[['Geography']]).toarray()
geo_encoded_df = pd.DataFrame(geo_encoded, columns=one_hot_encoder_geo.get_feature_names_out(['Geography']))

data = pd.concat([data.drop('Geography', axis=1), geo_encoded_df], axis=1)

X = data.drop('Exited', axis=1)
y = data['Exited']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

# # Save encoders and scaler for later use
# with open('artifacts/label_encoder_gender.pkl', 'wb') as file:
#     pickle.dump(label_encoder_gender, file)

# with open('artifacts/one_hot_encoder_geo.pkl', 'wb') as file:
#     pickle.dump(one_hot_encoder_geo, file)

# with open('artifacts/scaler.pkl', 'wb') as file:
#     pickle.dump(scaler, file)
# they are already saved in the other notebook

In [4]:
## Define a function to create the model and try different parameters(KerasClassifier)

def create_model(neurons=32,layers=1):
    model=Sequential()
    model.add(Dense(neurons,activation='relu',input_shape=(X_train.shape[1],)))

    for _ in range(layers-1):
        model.add(Dense(neurons,activation='relu'))

    model.add(Dense(1,activation='sigmoid'))
    model.compile(optimizer='adam',loss="binary_crossentropy",metrics=['accuracy'])

    return model

In [5]:
## Create a Keras classifier
model=KerasClassifier(layers=1,neurons=32,build_fn=create_model,verbose=1)

In [6]:
# Define the grid search parameters
param_grid = {
    'neurons': [16, 32, 64, 128],
    'layers': [1, 2],
    'epochs': [50, 100]
}

In [7]:
# Perform grid search
grid = GridSearchCV(estimator=model, param_grid=param_grid, n_jobs=-1, cv=3,verbose=1)
grid_result = grid.fit(X_train, y_train)

# Print the best parameters
print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))

Fitting 3 folds for each of 16 candidates, totalling 48 fits


/Users/kunalsharma/Documents/Bank_Churn_Classifier/venv/lib/python3.11/site-packages/scikeras/wrappers.py:925: UserWarning: ``build_fn`` will be renamed to ``model`` in a future release, at which point use of ``build_fn`` will raise an Error instead.
  X, y = self._initialize(X, y)
/Users/kunalsharma/Documents/Bank_Churn_Classifier/venv/lib/python3.11/site-packages/keras/src/layers/core/dense.py:93: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)
/Users/kunalsharma/Documents/Bank_Churn_Classifier/venv/lib/python3.11/site-packages/scikeras/wrappers.py:925: UserWarning: ``build_fn`` will be renamed to ``model`` in a future release, at which point use of ``build_fn`` will raise an Error instead.
  X, y = self._initialize(X, y)
/Users/kunalsharma/Documents/Bank_Churn_Classifier/venv/li

Epoch 1/50
Epoch 1/50
Epoch 1/50
Epoch 1/50
Epoch 1/50
Epoch 1/50
Epoch 1/50
Epoch 1/50
167/167 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - accuracy: 0.7598 - loss: 0.5317  
Epoch 2/50
167/167 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - accuracy: 0.6412 - loss: 0.6314
Epoch 2/50
167/167 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - accuracy: 0.5553 - loss: 0.6960
Epoch 2/50
167/167 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - accuracy: 0.7233 - loss: 0.5755
Epoch 2/50
167/167 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - accuracy: 0.6590 - loss: 0.6150
123/167 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.6220 - loss: 0.6408Epoch 2/50
167/167 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.6547 - loss: 0.61278
Epoch 2/50
167/167 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.4489 - loss: 0.8301
167/167 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.7372 - loss: 0.5518
Epoch 2/50
Epoch 2/50
167/167 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.8294 - loss: 0.4045  
Epoch 3/50
167/167 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.8008

/Users/kunalsharma/Documents/Bank_Churn_Classifier/venv/lib/python3.11/site-packages/scikeras/wrappers.py:925: UserWarning: ``build_fn`` will be renamed to ``model`` in a future release, at which point use of ``build_fn`` will raise an Error instead.
  X, y = self._initialize(X, y)
/Users/kunalsharma/Documents/Bank_Churn_Classifier/venv/lib/python3.11/site-packages/keras/src/layers/core/dense.py:93: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Epoch 1/50
84/84 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step   - accuracy: 0.8569 - loss: 0.3565
167/167 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.8572 - loss: 0.3435
167/167 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.8590 - loss: 0.3301
84/84 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step 
167/167 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.8663 - loss: 0.3111
Epoch 50/50
167/167 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.8570 - loss: 0.3398
32/84 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/stepEpoch 1/50acy: 0.8699 - loss: 0.2853 
34/84 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/stepep - accuracy: 0.8687 - loss: 0.29

...

100/167 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.8783 - loss: 0.3062Epoch 1/100
167/167 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.8679 - loss: 0.3191
Epoch 21/100
167/167 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.8987 - loss: 0.2446
Epoch 100/100
167/167 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.8622 - loss: 0.3158
 87/167 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.8710 - loss: 0.3328Epoch 12/100
167/167 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.8579 - loss: 0.3307
Epoch 13/100
167/167 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.8759 - loss: 0.3096
Epoch 20/100
167/167 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.8528 - loss: 0.3488 
Epoch 5/100
167/167 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.8684 - loss: 0.3331
Epoch 6/100
167/167 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.8869 - loss: 0.2553  
167/167 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.8759 - loss: 0.3097
Epoch 21/100
167/167 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.8752

/Users/kunalsharma/Documents/Bank_Churn_Classifier/venv/lib/python3.11/site-packages/scikeras/wrappers.py:925: UserWarning: ``build_fn`` will be renamed to ``model`` in a future release, at which point use of ``build_fn`` will raise an Error instead.
  X, y = self._initialize(X, y)
/Users/kunalsharma/Documents/Bank_Churn_Classifier/venv/lib/python3.11/site-packages/keras/src/layers/core/dense.py:93: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


 98/167 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.8660 - loss: 0.3213Epoch 1/100
167/167 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.8656 - loss: 0.3334
Epoch 7/100
167/167 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.8711 - loss: 0.3108
Epoch 23/100
167/167 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.8676 - loss: 0.3186
Epoch 14/100
167/167 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.8584 - loss: 0.3317
Epoch 22/100
167/167 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.8697 - loss: 0.3208 
Epoch 7/100
167/167 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.8763 - loss: 0.2993
Epoch 15/100
167/167 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.8724 - loss: 0.3139
 16/167 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.6423 - loss: 0.6368 Epoch 23/100
167/167 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.8645 - loss: 0.3298
Epoch 8/100
167/167 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.8738 - loss: 0.2996
Epoch 24/100
167/167 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accura

/Users/kunalsharma/Documents/Bank_Churn_Classifier/venv/lib/python3.11/site-packages/scikeras/wrappers.py:925: UserWarning: ``build_fn`` will be renamed to ``model`` in a future release, at which point use of ``build_fn`` will raise an Error instead.
  X, y = self._initialize(X, y)
/Users/kunalsharma/Documents/Bank_Churn_Classifier/venv/lib/python3.11/site-packages/keras/src/layers/core/dense.py:93: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Epoch 1/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 0s 396us/step - accuracy: 0.6497 - loss: 0.6392
Epoch 2/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 0s 351us/step - accuracy: 0.8054 - loss: 0.4508
Epoch 3/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 0s 349us/step - accuracy: 0.8162 - loss: 0.4247
Epoch 4/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 0s 353us/step - accuracy: 0.8220 - loss: 0.4147
Epoch 5/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 0s 354us/step - accuracy: 0.8351 - loss: 0.3948
Epoch 6/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 0s 349us/step - accuracy: 0.8453 - loss: 0.3887
Epoch 7/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 0s 356us/step - accuracy: 0.8466 - loss: 0.3759
Epoch 8/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 0s 357us/step - accuracy: 0.8500 - loss: 0.3662
Epoch 9/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 0s 350us/step - accuracy: 0.8579 - loss: 0.3584
Epoch 10/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 0s 347us/step - accuracy: 0.8520 - loss: 0.3604
Epoch 11/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 0s 337us/step - accuracy: 0.8553 - loss: 0.3530
Epoch 12/100
250/25

- Didn't include the hyperparameter tuning as it would require a large enough set of parameters and high computing power. This hyperparameter tuning is being done on a small set with same number of neurons in each layer, only for practice purpose